# Semantic Search using Cosine Similarity
Before running this Notebook, run 00_generate_embeddings.ipynb to generate the embeddings used for the semantic search

In [1]:
# Load Libraries
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Again we need the same model as in 00_generate_embeddings.ipynb to generate the embedding of the search term

In [2]:
# Set default (relative) directories
RAW_DATA_PATH = 'data/raw'
TRANSFORMED_DATA_PATH = 'data/transformed'

MODEL = 'paraphrase-multilingual-MiniLM-L12-v2' # https://www.sbert.net/docs/pretrained_models.html

model = SentenceTransformer(MODEL)
file_name = 'dataset_1.parquet'

In [3]:
# Read text and embeddings to search through
df = pd.read_parquet(f'{TRANSFORMED_DATA_PATH}/{file_name}')

Specify a search term and set a threshold for minimum cosine similarity. A fixed threshold is a navie approach and only used for simplicity

In [4]:
search_term = 'Ethik'
search_threshold = 0.3

Run cosine similarity search and print out sorted results

In [5]:
# Generate embedding of search term
search_embedding = model.encode([search_term], show_progress_bar=False)

# Convert embeddings column to NumPy array
embeddings_array = np.array(df['embeddings'].tolist())

# Calculate cosine similarity between search term embedding and all embeddings in the DataFrame
similarities = cosine_similarity(embeddings_array, search_embedding)

# Add similarities to DataFrame for simpler handling
df['cosine_similarity'] = similarities

# Sort texts according to cosine similarity with search term embedding
df = df.sort_values('cosine_similarity', ascending=False)

# Print out results
for idx, row in df[df['cosine_similarity'] >= search_threshold].iterrows():
    print(f'{row["pdf_name"]}- Page: {row["page_number"]} - Similarity: {row['cosine_similarity']}')
    print(row['chunk_text'])
    print('--------------------------')

fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 1214 - Similarity: 0.6205344873404243
So wie das Recht auf Wissen und Nicht-Wissen hier formuliert  wurde, haben wir haftungsrechtliche und ethische Bedenken:
--------------------------
fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 1232 - Similarity: 0.6205344873404243
So wie das Recht auf Wissen und Nicht-Wissen hier  formuliert wurde, haben wir haftungsrechtliche und ethische  Bedenken:
--------------------------
fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 1222 - Similarity: 0.6205344873404243
So wie das Recht auf Wissen und Nicht-Wissen hier formuliert  wurde, haben wir haftungsrechtliche und ethische Bedenken:
--------------------------
fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 539 - Similarity: 0.6189631384217519
ertes  fortuites alors que les règles cliniques et éthiques exigent  d’informer les personnes concernées.
--

In [6]:
# Example of a full text search with the search term in the DataFrame
for idx, row in df[df['chunk_text'].str.lower().str.contains(search_term.lower())].iterrows():
    print(f'{row["pdf_name"]}- Page: {row["page_number"]} - Similarity: {row['cosine_similarity']}')
    print(row['chunk_text'])
    print('--------------------------')

fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 2736 - Similarity: 0.5793962874028555
swissethics  Expertise der Mitglieder der Ethikkommissionen:  Da Gesuche zur Forschung mit Personen und zur Weiterverwendung von Daten und Proben heute einen Grossteil der Einreichungen bei den  Ethikkommissionen darstellen, ist eine Erweiterung der Expertise zur Beurteilung solcher Projekte (Art. 6 Abs. 1) zu begrüssen. Ebenfalls ist die  Expertise in Informationstechnologie im Gesundheitswesen heute sicherlich Standard bei der Beurteilung der Gesuche angesichts zunehmender  technischer Möglichkeiten im Ra
--------------------------
fedlex-data-admin-ch-eli-dl-proj-2023-5-cons_1-doc_12-de-pdf-a.pdf- Page: 44 - Similarity: 0.579396282587772
swissethics  Expertise der Mitglieder der Ethikkommissionen:  Da Gesuche zur Forschung mit Personen und zur Weiterverwendung von Daten und Proben heute einen Grossteil der Einreichungen bei den  Ethikkommissionen darstellen, ist eine Erwei